In [1]:

import pickle
import pandas as pd
import numpy as np
import sqlalchemy
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import json
import requests
import time
from IPython.display import display
import sys

In [2]:

url = "dbbikes.cvvkn6jkfbdf.eu-west-1.rds.amazonaws.com"
port = "3306"
db = "dbbikes"
user = "SEGroup25"
password = "SEGroup25?"



In [3]:

engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{db}", echo=True)


In [4]:

availability_sql = pd.read_sql("select * from dbbikes.availability", engine)

2023-04-18 10:10:26,435 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-18 10:10:26,437 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 10:10:26,802 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-18 10:10:26,804 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 10:10:26,813 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-18 10:10:26,815 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 10:10:26,848 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-18 10:10:26,852 INFO sqlalchemy.engine.Engine [generated in 0.00394s] ('dbbikes', 'select * from dbbikes.availability')
2023-04-18 10:10:26,871 INFO sqlalchemy.engine.Engine select * from dbbikes.availability
2023-04-18 10:10:26,872 INFO sqlalchemy.engine.Engine [raw sql] ()


In [5]:

weather_sql = pd.read_sql("select * from dbbikes.weather", engine)

2023-04-18 10:10:35,726 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-18 10:10:35,728 INFO sqlalchemy.engine.Engine [cached since 8.881s ago] ('dbbikes', 'select * from dbbikes.weather')
2023-04-18 10:10:35,745 INFO sqlalchemy.engine.Engine select * from dbbikes.weather
2023-04-18 10:10:35,746 INFO sqlalchemy.engine.Engine [raw sql] ()


In [6]:

weather_sql.shape[0]

2296

In [7]:

availability_sql.shape

(639425, 4)

In [8]:

availability_sql.head(5)

,available_bike_stands,available_bikes,last_update,number
0,23,7,2147483647,42
1,14,6,2147483647,30
2,21,12,2147483647,54
3,21,14,2147483647,108
4,14,16,2147483647,20


In [9]:

df_trimmed = availability_sql.groupby('number').tail(weather_sql.shape[0])


df_trimmed = df_trimmed.reset_index(drop=True)

In [10]:

df_trimmed.shape

(261897, 4)

In [11]:

df_trimmed['index'] = df_trimmed.groupby('number').cumcount()
merged_df = df_trimmed.join(weather_sql,on='index', how='left')
merged_df.head()

,available_bike_stands,available_bikes,last_update,number,index,time,cord_long,cord_lat,weather_main,main_temp,main_humidity,wind_speed,wind_degree,sys_sunset,sys_sunrise
0,0,1,2147483647,507,0,2023-03-30 13:53:20,-6.2672,53.344,Clouds,14.61,71.0,4.12,220.0,2023-03-30 18:54:48,2023-03-30 06:04:13
1,0,1,2147483647,507,1,2023-03-30 13:55:03,-6.2672,53.344,Clouds,14.61,71.0,4.12,220.0,2023-03-30 18:54:48,2023-03-30 06:04:13
2,0,1,2147483647,507,2,2023-03-30 14:11:04,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13
3,0,1,2147483647,507,3,2023-03-30 14:11:26,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13
4,0,1,2147483647,507,4,2023-03-30 14:15:02,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13


In [12]:

label_encoder = preprocessing.LabelEncoder()
merged_df['weather_main'] = label_encoder.fit_transform(merged_df['weather_main'])

In [13]:

merged_df.tail()

,available_bike_stands,available_bikes,last_update,number,index,time,cord_long,cord_lat,weather_main,main_temp,main_humidity,wind_speed,wind_degree,sys_sunset,sys_sunrise
261892,20,0,2147483647,39,2295,2023-04-18 09:10:03,-6.2672,53.344,1,10.93,86.0,5.14,90.0,2023-04-18 19:29:08,2023-04-18 05:19:25
261893,27,13,2147483647,83,2295,2023-04-18 09:10:03,-6.2672,53.344,1,10.93,86.0,5.14,90.0,2023-04-18 19:29:08,2023-04-18 05:19:25
261894,32,7,2147483647,92,2295,2023-04-18 09:10:03,-6.2672,53.344,1,10.93,86.0,5.14,90.0,2023-04-18 19:29:08,2023-04-18 05:19:25
261895,0,30,2147483647,21,2295,2023-04-18 09:10:03,-6.2672,53.344,1,10.93,86.0,5.14,90.0,2023-04-18 19:29:08,2023-04-18 05:19:25
261896,23,7,2147483647,88,2295,2023-04-18 09:10:03,-6.2672,53.344,1,10.93,86.0,5.14,90.0,2023-04-18 19:29:08,2023-04-18 05:19:25


In [14]:

merged_df['month'] = merged_df['time'].dt.month
merged_df['dayofweek'] = merged_df['time'].dt.dayofweek
merged_df['hour'] = merged_df['time'].dt.hour
merged_df['minute'] = merged_df['time'].dt.minute

In [15]:
print(merged_df.isnull().sum().sum())

0


In [16]:

merged_df.dtypes

available_bike_stands             int64
available_bikes                   int64
last_update                       int64
number                            int64
index                             int64
time                     datetime64[ns]
cord_long                       float64
cord_lat                        float64
weather_main                      int64
main_temp                       float64
main_humidity                   float64
wind_speed                      float64
wind_degree                     float64
sys_sunset               datetime64[ns]
sys_sunrise              datetime64[ns]
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

In [17]:
merged_df.shape

(261897, 19)

In [18]:

X = merged_df[['number','month','hour','minute','weather_main','main_temp','main_humidity','wind_speed','dayofweek']]
Y = merged_df.available_bikes
X_train, X_test,y_train,y_test = train_test_split(X, Y, test_size=0.3,random_state=0)
bikes_model = RandomForestRegressor(n_estimators=15)

bikes_model.fit(X_train, y_train)
y_pred = bikes_model.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})


print('Test Set Predictions for Station: ')
print(df)
print()


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


with open('bikes_model.pkl', 'wb') as handle:
    pickle.dump(bikes_model, handle, pickle.HIGHEST_PROTOCOL)

Test Set Predictions for Station: 
        Actual  Predicted
255411      27  27.000000
112239       0   0.400000
11859       12  16.133333
65541        2   2.000000
73006        3   2.000000
...        ...        ...
114417       0   0.000000
230295      11  11.600000
171287       7   8.400000
36795        4   3.600000
171468       8   6.400000

[78570 rows x 2 columns]

Mean Absolute Error: 1.00694836875822
Mean Squared Error: 4.855455319389646
Root Mean Squared Error: 2.2035097729280997
The accuracy score :  0.9435041067771579


In [19]:
A = merged_df[['number','month','hour','minute','weather_main','main_temp','main_humidity','wind_speed','dayofweek']]
B = merged_df.available_bike_stands
A_train, A_test,b_train,b_test = train_test_split(A, B, test_size=0.3,random_state=0)
stands_model = RandomForestRegressor(n_estimators=15)

stands_model.fit(X_train, b_train)
b_pred =stands_model.predict(A_test)
df = pd.DataFrame({'Actual': b_test, 'Predicted': b_pred})

print('Test Set Predictions for Station: ')
print(df)
print()

#Accuracy score
print('Mean Absolute Error:', metrics.mean_absolute_error(b_test, b_pred))
print('Mean Squared Error:', metrics.mean_squared_error(b_test, b_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(b_test, b_pred)))
print ("The accuracy score : ",metrics.r2_score(b_test, b_pred))

with open('stands_model.pkl', 'wb') as handle:
    pickle.dump(stands_model, handle, pickle.HIGHEST_PROTOCOL)

Test Set Predictions for Station: 
        Actual  Predicted
255411       3   3.266667
112239      40  39.666667
11859       28  26.533333
65541       25  25.000000
73006       27  26.400000
...        ...        ...
114417      30  30.000000
230295      17  15.933333
171287      32  29.333333
36795       26  25.333333
171468      32  33.200000

[78570 rows x 2 columns]

Mean Absolute Error: 1.002229010224428
Mean Squared Error: 4.837800404451797
Root Mean Squared Error: 2.1995000351106606
The accuracy score :  0.9583222575125359
